Importanje knjiznjic

In [14]:
import tensorflow as tf
import keras as kr
import tf_keras as tfk
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import sklearn

# Check TensorFlow version and GPU availability
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {kr.__version__}")
print(f"Tf-keras Version: {tfk.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"MediaPipe Version: {mp.__version__}")
print(f"Scikit-learn Version: {sklearn.__version__}")
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow Version: 2.16.2
Keras Version: 3.10.0
Tf-keras Version: 2.16.0
NumPy Version: 1.26.4
Pandas Version: 2.2.3
OpenCV Version: 4.11.0
MediaPipe Version: 0.10.21
Scikit-learn Version: 1.4.2
Num GPUs Available: 1


In [16]:
import tensorflow_hub as hub

print(f"TensorFlow Hub Version: {hub.__version__}")

TensorFlow Hub Version: 0.16.1


In [17]:
# Zdaj lahko uporabljas hub.KerasLayer
model_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"
base_model = hub.KerasLayer(model_url,
                            input_shape=(224, 224, 3),
                            trainable=False) # Trainable=False, kot smo se dogovorili za hitro treniranje na uporabnika

# ... in gradis svoj Sequential model na vrhu base_modela.